### Student Information
Name:  李博業

Student ID: 410978058

GitHub ID: BORYA-dev

Kaggle name:

Kaggle private scoreboard snapshot:

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home exercises** in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook.


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking:
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (0.6N + 1 - x) / (0.6N) * 10 + 20 points, where N is the total number of participants, and x is your rank. (ie. If there are 100 participants and you rank 3rd your score will be (0.6 * 100 + 1 - 3) / (0.6 * 100) * 10 + 20 = 29.67% out of 30%.)   
    Submit your last submission **BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developing the model for the competition (You can use code and comment on it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained.


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th, 11:59 pm, Tuesday)__.

- emotion
  - The emotion(answer) of data
    - sadness
    - disgust
    - anticipation
    - joy
    - trust
    - anger
    - fear
    - surprise

- data_identification :
  - identify the id is train or test
  
- tweets_DM.json : Main Data
  - type : list
  - length = 1867535
  - record :
    {
    '_score': 391,
    '_index': 'hashtag_tweets',
    '_source': {'tweet': {'hashtags': ['Snapchat'],
      'tweet_id': '0x376b20',
      'text': 'People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that\'s <LH>'}},
    '_crawldate': '2015-05-23 11:42:47',
    '_type': 'tweets'
    }

In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split

import nltk
import re
import stringprep
nltk.download('punkt_tab')
# nltk.download('wordnet')

import pickle

# Random Forest
from sklearn.ensemble import RandomForestClassifier

# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.metrics import accuracy_score

# NN
import keras
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import ReLU, Softmax

from sklearn.preprocessing import LabelEncoder


## BERT Embedding + Fine Tuning
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset,DataLoader
from tqdm.auto import tqdm
import torch.utils.data as data

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

- Prepare Data

In [ ]:
### Begin Assignment Here

### 準備資料

file_path = '/content/drive/MyDrive/Data_Lab2_HW/tweets_DM.json'

data = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line_number, line in enumerate(file, start=1):
        try:
            data.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error in line {line_number}: {e}")


emotion_data = pd.read_csv('/content/drive/MyDrive/Data_Lab2_HW/emotion.csv')
data_id = pd.read_csv('/content/drive/MyDrive/Data_Lab2_HW/data_identification.csv')

# 生成DataFrame

temp = []
for sub_d in data:
  temp.append(sub_d['_source']['tweet'])
data_df = pd.DataFrame(temp)


data_df = data_df.merge(data_id, on = "tweet_id", how="left")
data_df = data_df.merge(emotion_data, on = "tweet_id", how = "left")

data_df.head(5)


In [ ]:
# 訓練集、測試集
training_data = data_df.query("identification == 'train'").reset_index(drop =True)
testing_data = data_df.query("identification == 'test'").reset_index(drop =True)

- Sample

In [ ]:
# 抽等量

#  min_class_num = training_data.emotion.value_counts().min()

# 訓練集抽樣
# sampled_training = training_data.groupby('emotion').apply(lambda x: x.sample(n=min_class_num, random_state = 42) )
# sampled_training = sampled_training.reset_index(drop=True)
# x_train = sampled_training.drop(columns = "emotion")
# y_train = sampled_training.emotion


# # 分層抽樣
# # Strafied sampling
# x_train, x_test, y_train, y_test = train_test_split(
#     x_train, y_train, test_size=0.2, random_state=42, stratify=y_train
# )

# x_train.reset_index(drop=True, inplace = True)
# y_train.reset_index(drop=True, inplace = True)
# x_test.reset_index(drop=True, inplace=True)
# y_test.reset_index(drop=True, inplace=True)

<ipython-input-5-304cfdb7fe4c>:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_training = training_data.groupby('emotion').apply(lambda x: x.sample(n=min_class_num, random_state = 42) )


In [ ]:
# 先掠過

train_data_sample = training_data.sample(frac=0.25, random_state=20)
train_data_sample = train_data_sample.reset_index(drop=True)

x_train = train_data_sample.drop(columns = ["emotion", "identification"])
y_train = train_data_sample.emotion

# 分層抽樣
# Strafied sampling
x_train, x_test, y_train, y_test = train_test_split(
    x_train, y_train, test_size=0.2, random_state=20, stratify=y_train
)

x_train.reset_index(drop=True, inplace = True)
y_train.reset_index(drop=True, inplace = True)
x_test.reset_index(drop=True, inplace = True)
y_test.reset_index(drop=True, inplace = True)

- Pre-processing

In [19]:
emoji_dict = {
    '😂': '[joy]',
    '❤️': '[love]',
    '😍': '[adoration]',
    '😭': '[cry]',
    '❤': '[care]',
    '😊': '[happy]',
    '🙏': '[pray]',
    '😘': '[kiss]',
    '💕': '[love_each_other]',
    '🔥': '[fire]',
    '😩': '[weary]',
    '🤔': '[think]',
    '💯': '[perfect]',
    '💙': '[loyalty]',
    '🙄': '[annoyed]',
    '😁': '[happy]',
    '🙌': '[celebrate]',
    '🙏🏾': '[pray]',
    '👍': '[approve]',
    '🙏🏽': '[pray]',
    '♡':'[love]',
    '🕘':'[time]',
    '🙋':'[hi]',
    '😃': '[smile]'
}

In [ ]:
# emoji
def trans_emoji(text):
  for key, value in emoji_dict.items():
    text = text.replace(key, value)
  return text

def cleaning(text):

    text = text.replace("<lh>", "")
    text = text.replace("<LH>", "")

    text = trans_emoji(text)

    text = re.sub(r'[@#$%^&*()_+{}[\]:;"\'<>,.~\\/\-|=]', '', text)

    # remove some common words
    text = re.sub(r'\b(?:and|is|in|the|on|of|to|for)\b', '', text, flags=re.IGNORECASE)
    # remove numbers
    text = re.sub(r'\b\d+\b', '', text)
    # remove blank space
    text = re.sub(r'\s+', ' ', text).strip() # \s+ 一個或多個空白字符
    # remove links
    text = re.sub('((www.[^s]+)|(https?://[^s]+))',' ',text)
    return text



# Stemming
# st = nltk.PorterStemmer()

# def stemming_on_text(data):
#     '''
#     input should be tokenized words : list
#     '''
#     text = [st.stem(word) for word in data]
#     return text

# # Lemmatization
# lm = nltk.WordNetLemmatizer()
# def lemmatizer_on_text(data):
#     text = [lm.lemmatize(word) for word in data]
#     return text


def pre_processing(data):

  '''
  data:DataFrame with text
  '''

  dataset = data['text']


  # 轉小寫
  dataset = dataset.apply(lambda text: text.lower().strip())

  # 移除 removing

  # original tokens
  only_tokens = dataset.apply(lambda x: nltk.word_tokenize(x) )
  only_tokens.name = "tokens"

  # cleaning
  dataset = dataset.apply(lambda x: cleaning(x) )
  dataset.name = "pre_processing_text"

  # cleaning + tokens
  tt = dataset.apply(lambda x: nltk.word_tokenize(x))
  tt.name = "pre_processing_tokens"


  pre_df = pd.concat([pd.DataFrame(only_tokens), pd.DataFrame(dataset),pd.DataFrame(tt)], axis=1)

  return pre_df

In [ ]:
# Combine tje columns with pro-processing & original text

x_train = pd.concat([x_train, pre_processing(x_train)], axis=1)
x_train.reset_index(drop=True, inplace=True)

x_test = pd.concat([x_test, pre_processing(x_test)], axis=1)
x_test.reset_index(drop=True, inplace=True)

testing_data = pd.concat([testing_data, pre_processing(testing_data)], axis=1)
testing_data.reset_index(drop=True, inplace=True)

In [ ]:
x_train.head(5)

,hashtags,tweet_id,text,tokens,pre_processing_text,pre_processing_tokens
0,[God],0x231c61,Not the ones that want commitment... patiently...,"[not, the, ones, that, want, commitment, ..., ...",not ones that want commitment patiently waitin...,"[not, ones, that, want, commitment, patiently,..."
1,"[everyday, special]",0x330e2f,31 Push the <LH> #everyday. Be some #special. ...,"[31, push, the, <, lh, >, #, everyday, ., be, ...",push everyday be some special august at 0515pm,"[push, everyday, be, some, special, august, at..."
2,[],0x30eca8,@AADaddario @lotstar @michaellonghair loved to...,"[@, aadaddario, @, lotstar, @, michaellonghair...",aadaddario lotstar michaellonghair loved watch...,"[aadaddario, lotstar, michaellonghair, loved, ..."
3,"[payforplay, thatmeansyou, scum]",0x20e851,I AM IN <LH> THAT MEANS Y O U P A Y M E FOR AN...,"[i, am, in, <, lh, >, that, means, y, o, u, p,...",i am that means y o u p a y m e any servise i ...,"[i, am, that, means, y, o, u, p, a, y, m, e, a..."
4,[],0x254bff,Father God you are faithful. Be Lord of my lif...,"[father, god, you, are, faithful, ., be, lord,...",father god you are faithful be lord my life today,"[father, god, you, are, faithful, be, lord, my..."


- TFIDF + Random Forest

In [ ]:
# tweet_tokenizer = nltk.tokenize.TweetTokenizer() # 本來有用 max_df=0.9, tokenizer = tweet_tokenizer.tokenize
TF_IDF_vectorizer = TfidfVectorizer(max_features=1000) # 3000 2500 2000 都會爆
output_vectorizer = TF_IDF_vectorizer.fit_transform(x_train.text.values)  # fit + transform

In [ ]:
output_vectorizer_test = TF_IDF_vectorizer.transform(x_test.text.values)  # only transform

In [ ]:
output_vectorizer_true_test = TF_IDF_vectorizer.transform(testing_data.text.values)  # only transform

In [ ]:
# 寫出

#訓練+測試
with open('/content/drive/MyDrive/Data_Lab2_HW/tf-idf-trained.pkl', 'wb') as file:  # 'wb' 表示以二進位寫入模式
    pickle.dump([TF_IDF_vectorizer, output_vectorizer, y_train, output_vectorizer_test, y_test], file)

# 真正測試資料
with open('/content/drive/MyDrive/Data_Lab2_HW/testing_data.pkl', 'wb') as file:  # 'wb' 表示以二進位寫入模式
    pickle.dump([testing_data, output_vectorizer_true_test], file)


In [ ]:
#讀入

with open('/content/drive/MyDrive/Data_Lab2_HW/tf-idf-trained.pkl', 'rb') as file:  # 'rb' 表示以二進位讀取模式
    tfidf_list = pickle.load(file)

with open('/content/drive/MyDrive/Data_Lab2_HW/testing_data.pkl', 'rb') as file:  # 'rb' 表示以二進位讀取模式
    testing_list = pickle.load(file)

In [ ]:
X = tfidf_list[1]
y_train = tfidf_list[2].values

In [ ]:
model = RandomForestClassifier()

## training!
DT_model = model.fit(output_vectorizer, y_train)

## predict! training

In [ ]:
from joblib import dump, load

In [ ]:
# 寫出模型
dump(DT_model, '/content/drive/MyDrive/Data_Lab2_HW/random_forest_3000.joblib')  # 儲存模型

['/content/drive/MyDrive/Data_Lab2_HW/random_forest_3000.joblib']

In [ ]:
# 讀入模型
DT_model = load('/content/drive/MyDrive/Data_Lab2_HW/random_forest.joblib')

In [ ]:
## Training Accuracy

y_train_pred = DT_model.predict(output_vectorizer)

acc_train = accuracy_score(y_true=y_train, y_pred=y_train_pred)

print('training accuracy: {}'.format(round(acc_train, 2)))

training accuracy: 0.96


In [ ]:
X = tfidf_list[3]

In [ ]:
# Mini-Testing
# output_vectorizer_testing = TF_IDF_vectorizer.fit_transform(mini_X_test['preprocess_text'])

# Predict Mini-Testing (by Random Forest)
y_test_result = DT_model.predict( output_vectorizer_test )

acc_train = accuracy_score(y_true= y_test, y_pred= y_test_result)

print('Mini-testing accuracy: {}'.format(round(acc_train, 2)))

Mini-testing accuracy: 0.41


In [ ]:
DT_model = load('/content/drive/MyDrive/Data_Lab2_HW/random_forest.joblib')

In [ ]:
y_test_result = DT_model.predict( output_vectorizer_true_test )

In [ ]:
# 寫出結果
result = pd.DataFrame( {
   "id" : testing_data.tweet_id,
   "emotion" : y_test_result
} )
result.to_csv("/content/drive/MyDrive/submission_test_v9.csv", index = 0)
result.head(5)

,id,emotion
0,0x1c7f0f,disgust
1,0x1c7f12,anger
2,0x1c7f13,trust
3,0x1c7f17,trust
4,0x1c7f18,anger


- TFIDF + NN

- Word2Vec

In [ ]:
train_corpus = x_train.pre_processing_tokens.values

In [ ]:
# Training Word2Vec

from gensim.models import Word2Vec

# setting
vector_dim = 100
window_size = 5
min_count = 1
workers = 2

word2vec_model = Word2Vec(sentences=train_corpus,
              vector_size = vector_dim,
              window=window_size,
              min_count=min_count,
              workers = workers)

In [ ]:
def mean_word2vec(text_list):
  result = np.zeros(vector_dim, dtype = np.float64)
  len = 0
  for i in text_list:
    if i in word2vec_model.wv:
      result = result + word2vec_model.wv[i]
      len = len + 1

  return result/len if len != 0 else  result

In [ ]:
x_train["word2vec_100"] = x_train.apply(lambda x: mean_word2vec(x['tokens']), axis = 1)
x_test["word2vec_100"] = x_test.apply(lambda x: mean_word2vec(x['tokens']), axis = 1)
testing_data["word2vec_100"] = testing_data.apply(lambda x: mean_word2vec(x['tokens']), axis = 1)

In [ ]:
# LABEL Y
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

In [ ]:
# Training Random Forest

X = list(x_train.word2vec_100.values)

model = RandomForestClassifier()

## training!
DT_model = model.fit(X, y_train_encoded)

In [ ]:
## Testing Accuracy

x_test["word2vec_100"] = x_test.apply(lambda x: mean_word2vec(x['tokens']), axis = 1)
X = list(x_test.word2vec_100.values)

y_test_pred = DT_model.predict(X)

acc_train = accuracy_score(y_true = y_test_encoded, y_pred = y_test_pred)

print('testing accuracy: {}'.format(round(acc_train, 2)))

training accuracy: 0.47


In [ ]:
# Prediction

X = list(testing_data.word2vec_100.values)

y_test_result = DT_model.predict(X)

y_test_result = le.inverse_transform( y_test_result )

result = pd.DataFrame( {
   "id" : testing_data.tweet_id,
   "emotion" : y_test_result
} )

result.to_csv("/content/drive/MyDrive/submission_test_v4.csv", index = 0)
result.head(5)

,id,emotion
0,0x28b412,joy
1,0x2de201,anticipation
2,0x218443,sadness
3,0x2939d5,joy
4,0x26289a,anticipation


- NN

In [ ]:
# One Hot Encoding

## deal with label (string -> one-hot)



label_encoder = LabelEncoder()
label_encoder.fit(y_train)

print('check label: ', label_encoder.classes_)
print('\n## Before convert')
print('y_train[0:4]:\n', y_train[0:4])
print('\ny_train.shape: ', y_train.shape)
print('y_test.shape: ', y_test.shape)

def label_encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def label_decode(le, one_hot_label):
    dec = np.argmax(one_hot_label, axis=1)
    return le.inverse_transform(dec)

y_train_one_hot = label_encode(label_encoder, y_train)
y_test_one_hot = label_encode(label_encoder, y_test)

print('\n\n## After convert')
print('y_train[0:4]:\n', y_train_one_hot[0:4])
print('\ny_train.shape: ', y_train_one_hot.shape)
print('y_test.shape: ', y_test_one_hot.shape)



check label:  ['anger' 'anticipation' 'disgust' 'fear' 'joy' 'sadness' 'surprise'
 'trust']

## Before convert
y_train[0:4]:
 0    anticipation
1             joy
2             joy
3         sadness
Name: emotion, dtype: object

y_train.shape:  (349335,)
y_test.shape:  (87334,)


## After convert
y_train[0:4]:
 [[0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]]

y_train.shape:  (349335, 8)
y_test.shape:  (87334, 8)


In [ ]:
# I/O size check
input_shape = output_vectorizer.shape[1]
output_shape = len(label_encoder.classes_)

In [ ]:
# input layer
model_input = Input(shape=(input_shape, ))  # 500
X = model_input

# 1st hidden layer
X_W1 = Dense(units=64)(X)  # 64
H1 = ReLU()(X_W1)

# 2nd hidden layer
H1_W2 = Dense(units=64)(H1)  # 64
H2 = ReLU()(H1_W2)

# output layer
H2_W3 = Dense(units=output_shape)(H2)  # 4
H3 = Softmax()(H2_W3)

model_output = H3

# create model
model = Model(inputs=[model_input], outputs=[model_output])

# loss function & optimizer
model.compile(optimizer='adam',
       loss='categorical_crossentropy',
       metrics=['accuracy'])

# show model construction
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 1000)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          64,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu (ReLU)                         │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ re_lu_1 (ReLU)                       │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 8)                   │             520 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ softmax (Softmax)                    │ (None, 8)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 68,744 (268.53 KB)

 Trainable params: 68,744 (268.53 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
np.array(x_train.word2vec_100)

(349335,)

In [ ]:
y_train_one_hot.shape

(349335, 8)

In [ ]:
from keras import backend as K
x_train_nn = K.cast_to_floatx(x_train.word2vec_100.values)
y_train_nn = K.cast_to_floatx(y_train_one_hot)

AttributeError: module 'keras.api.backend' has no attribute 'cast_to_floatx'

In [ ]:
import tensorflow as tf

In [ ]:
tf.convert_to_tensor(x_train.word2vec_100.values[0])

<tf.Tensor: shape=(100,), dtype=float64, numpy=
array([-0.28891157, -0.51392198, -1.2580553 , -0.07961606,  0.78792557,
       -1.01779029, -0.4315294 , -0.3566817 ,  0.1275314 , -0.93393448,
       -0.97445206, -0.52523305,  0.46085551,  0.01323835,  1.11242473,
       -0.11519837, -2.39290107, -0.137721  ,  1.55165731,  0.3262168 ,
        0.31253075, -0.40876732, -1.04998667, -0.31491014,  0.14218648,
        0.32194564, -0.24623724, -1.36981471,  0.07970582,  0.60668447,
        0.92419443,  0.60009167,  0.30457177,  0.5232327 , -0.77669291,
        1.29563875, -0.90363368, -0.1206033 , -0.55609911, -1.11152058,
        1.41376781, -1.31118262,  0.8427708 ,  0.28585574,  0.62808239,
        0.59105415, -0.03813672, -0.90437077,  0.0426878 ,  0.06495104,
        0.89675321, -1.12385925, -0.11752596, -1.59556571, -0.36668717,
        0.69645377, -1.07754941,  0.83813685,  0.50564614,  0.19888692,
        0.24486466, -0.42000695,  0.1279143 ,  1.61156959,  0.14878466,
        0.516757

In [ ]:
x_train.word2vec_100.values.astype(dtype = [(np.float64)])

TypeError: Field elements must be 2- or 3-tuples, got '<class 'numpy.float64'>'

In [ ]:
# Train
# from keras.callbacks import CSVLogger
# csv_logger = CSVLogger('logs/training_log.csv')

# training setting
epochs = 25
batch_size = 32

# training!
history = model.fit(output_vectorizer, y_train_one_hot,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data = (output_vectorizer_test, y_test_one_hot))
print('training finish')

Epoch 1/25
10917/10917 ━━━━━━━━━━━━━━━━━━━━ 46s 4ms/step - accuracy: 0.4612 - loss: 1.4912 - val_accuracy: 0.4957 - val_loss: 1.3904
Epoch 2/25
10917/10917 ━━━━━━━━━━━━━━━━━━━━ 49s 5ms/step - accuracy: 0.5043 - loss: 1.3691 - val_accuracy: 0.5030 - val_loss: 1.3706
Epoch 3/25
10917/10917 ━━━━━━━━━━━━━━━━━━━━ 78s 4ms/step - accuracy: 0.5194 - loss: 1.3271 - val_accuracy: 0.5049 - val_loss: 1.3612
Epoch 4/25
10917/10917 ━━━━━━━━━━━━━━━━━━━━ 83s 4ms/step - accuracy: 0.5274 - loss: 1.3029 - val_accuracy: 0.5068 - val_loss: 1.3592
Epoch 5/25
10917/10917 ━━━━━━━━━━━━━━━━━━━━ 48s 4ms/step - accuracy: 0.5364 - loss: 1.2818 - val_accuracy: 0.5056 - val_loss: 1.3592
Epoch 6/25
10917/10917 ━━━━━━━━━━━━━━━━━━━━ 80s 4ms/step - accuracy: 0.5419 - loss: 1.2670 - val_accuracy: 0.5012 - val_loss: 1.3685
Epoch 7/25
10917/10917 ━━━━━━━━━━━━━━━━━━━━ 83s 4ms/step - accuracy: 0.5466 - loss: 1.2508 - val_accuracy: 0.5030 - val_loss: 1.3667
Epoch 8/25
10917/10917 ━━━━━━━━━━━━━━━━━━━━ 47s 4ms/step - accuracy: 

In [ ]:
## predict
pred_result_test = model.predict(output_vectorizer_test, batch_size=128)
pred_result_test[:5]

683/683 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step


array([[7.8271632e-04, 1.0990217e-01, 4.6986011e-03, 7.5955922e-03,
        6.0794413e-01, 2.6080755e-03, 2.5845398e-03, 2.6388413e-01],
       [3.3740713e-03, 9.8723710e-02, 3.4478102e-02, 1.4036582e-01,
        5.9580362e-01, 2.9223016e-02, 2.6972406e-02, 7.1059316e-02],
       [3.4210362e-02, 1.4242639e-01, 1.5957287e-01, 1.6857922e-02,
        2.0275207e-01, 2.7755481e-01, 7.5862952e-02, 9.0762660e-02],
       [3.1009625e-10, 1.8502025e-08, 8.4229852e-11, 2.8260108e-11,
        9.9999994e-01, 3.6594061e-10, 4.7375298e-10, 1.4810303e-08],
       [9.5373005e-02, 5.8587644e-02, 3.9580223e-01, 2.4893282e-02,
        1.2639330e-01, 2.5650522e-01, 2.1018080e-02, 2.1427071e-02]],
      dtype=float32)

In [ ]:
pred_result_test = label_decode(label_encoder, pred_result_test)
pred_result_test[:5]

array(['joy', 'joy', 'sadness', 'joy', 'disgust'], dtype=object)

In [ ]:
print(f'testing accuracy: { round(accuracy_score(y_test.values, pred_result_test),2) }')

testing accuracy: 0.49


In [ ]:
## predict
pred_result_test = model.predict(output_vectorizer_true_test, batch_size=128)
pred_result_test = label_decode(label_encoder, pred_result_test)
pred_result_test[:5]

3219/3219 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step


array(['anticipation', 'anticipation', 'joy', 'joy', 'anticipation'],
      dtype=object)

In [ ]:
result = pd.DataFrame( {
   "id" : testing_data.tweet_id,
   "emotion" : pred_result_test
} )

result.to_csv("/content/drive/MyDrive/submission_test_v4.csv", index = 0)
result.head(5)

,id,emotion
0,0x28b412,anticipation
1,0x2de201,anticipation
2,0x218443,joy
3,0x2939d5,joy
4,0x26289a,anticipation


- BERT Embedding
  - 所需Input
    - token id : 每個單字在字典裡對應的一個 index\
    - attention mask : padding成相同長度後，attention mask 讓self-attention知道哪些是該注意的字
    - segment id : 如果一次Input兩個句子, segment id 區分兩個句子的Index。

In [6]:
# Labeling

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

# y_train
y_train_trans = label_encoder.transform(y_train)  # +1 避免label有 0 # 不然會有CUDA error

# y_test
y_test_trans = label_encoder.transform(y_test)

print( f"After labeling : {y_train_trans}")

After labeling : [6 4 4 ... 7 4 1]


In [7]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [8]:
# import 相關套件 & download tokenizers and model

import torch
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'AdamW' is not defined

In [30]:
y_train_trans.max()

7

In [14]:
# 可以跳過 這只是 pre_trained tokenizer 內部的步驟

# 1. tokenize sentence

sentence = training_data.text[0]
print(f"Original_text : {sentence}")
tokens = tokenizer.tokenize(sentence)
print(f"Tokenized_text : {tokens}")

# 2. Add [CLS] and [SEP] tokens
tokens = ['CLS'] + tokens + ['SEP']

# 3. Padding the input 每個句子padding成一樣的長度

T = 45 # 給定長度]

padded_tokens = tokens + ['[PAD]' for _ in range(0,T-len(tokens))]
# attention_mask padding成相同長度後，讓self-attention layer 知道哪些是padding 哪些是要關注的字
attn_mask = [ 1 if i != '[PAD]' else 0 for i in padded_tokens]

seg_ids = [ 0 for _ in range(0, len(padded_tokens))]

sent_ids = tokenizer.convert_tokens_to_ids(padded_tokens)

print(f"Tokens id : {sent_ids}")
print(f"Attention mask : {attn_mask}")
print(f"Segment_id : {seg_ids}")

# * 4. Input tensor 轉tensor
# token_ids = torch.tensor(sent_ids).unsqueeze(0)
# attn_mask = torch.tensor(attn_mask).unsqueeze(0)
# seg_ids   = torch.tensor(seg_ids).unsqueeze(0)

# print(f"Tensor-Tokens id : {token_ids}")
# print(f"Tensor-Attention mask : {attn_mask}")
# print(f"Tensor-Segment_id : {seg_ids}")

Original_text : People who post "add me on #Snapchat" must be dehydrated. Cuz man.... that's <LH>
Tokenized_text : ['people', 'who', 'post', '"', 'add', 'me', 'on', '#', 'snap', '##cha', '##t', '"', 'must', 'be', 'de', '##hy', '##dra', '##ted', '.', 'cu', '##z', 'man', '.', '.', '.', '.', 'that', "'", 's', '<', 'l', '##h', '>']
Tokens id : [100, 2111, 2040, 2695, 1000, 5587, 2033, 2006, 1001, 10245, 7507, 2102, 1000, 2442, 2022, 2139, 10536, 7265, 3064, 1012, 12731, 2480, 2158, 1012, 1012, 1012, 1012, 2008, 1005, 1055, 1026, 1048, 2232, 1028, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Attention mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Segment_id : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Tensor-Tokens id : tensor([[  100,  2111,  2040,  2695,  1000,  5587,  2033,  2006,  1001, 10245,
          7507, 

In [1]:
tokenizer(training_data.text[0], add_special_tokens=True, truncation=True,
     padding='max_length', return_tensors="pt")

NameError: name 'tokenizer' is not defined

In [16]:
# model inference

output = model( input_ids = token_ids, attention_mask = attn_mask, token_type_ids = seg_ids)

last_hidden_state, pooler_output = output[0], output[1]

# last_hidden_state: hiddenstate of each tokens
# pooler_output: sentence embedding 就是句子開頭的[CLS](代表該句子的摘要信息)

print(f"last_hidden_state shape:{last_hidden_state.shape}") # [batch_size, sequence_length, hidden_size]
print(f"pooler_output shape:{pooler_output.shape}") # [batch_size, hidden_size]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [9]:
# 進Pytorch前 Dataset定義

class TweetData(Dataset):  # Dataset 是 torch 來的

  def __init__(self, inputs, targets, tokenizer, max_len): # 處理資料讀取
    self.texts = inputs
    self.labels = targets
    self.tokenizers = tokenizer
    self.len = max_len


  def __len__(self):  # 回傳dataset長度

    return len(self.texts)

  def __getitem__(self, index): # 特定資料回傳 # 可做数据增强、normalize

    sentence = self.texts[index]

    t = self.tokenizers(sentence, max_length = self.len, add_special_tokens=True, truncation=True,
       padding='max_length', return_tensors="pt")

    result = {
       'input_ids' : t['input_ids'].squeeze(0),
       'token_type_ids': t['token_type_ids'].squeeze(0), # 要squeeze(0) 不然維度少一個報錯
       'attention_mask': t['attention_mask'].squeeze(0),
       'labels' : torch.tensor( self.labels[index], dtype = torch.long )
    }

    return result


In [11]:
# Dataloader
max_len = 128
train_loader = DataLoader(TweetData(x_train.text.tolist(), y_train_trans, tokenizer, max_len), batch_size=32, shuffle=True, num_workers=1) # new_workers 用多少線程處理
test_loader = DataLoader(TweetData(x_test.text.tolist(), y_test_trans, tokenizer, max_len ), batch_size =32, shuffle=True, num_workers=1)

1

In [10]:
t = 0
tt = tqdm(train_loader)
for data in tt:
  t = t+1
  print(data)
  if t > 1 :
    break

  0%|          | 0/4549 [00:00<?, ?it/s]

{'input_ids': tensor([[  101,  1030, 24264,  ...,     0,     0,     0],
        [  101,  1045,  2293,  ...,     0,     0,     0],
        [  101,  2131,  7629,  ...,     0,     0,     0],
        ...,
        [  101,  1030,  2585,  ...,     0,     0,     0],
        [  101,  2067,  2013,  ...,     0,     0,     0],
        [  101,  1030,  5925,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([3, 6, 6, 5, 6, 2, 8, 5, 6, 5, 6, 3, 7, 3, 6, 8, 3, 7, 2, 2, 5, 5, 2, 2,
        8, 2, 8, 6, 5, 3, 3, 5, 1, 5, 5, 5, 4, 6, 5, 5, 6, 

- SequenceClassifierOutput(loss=tensor(0.4921, device='cuda:0', grad_fn= NllLossBackward0 ), logits=tensor([[-0.0726, -0.5257]], device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)




In [ ]:
optimizer = torch.optim.AdamW(params = model.parameters(), lr = 1e-4)

for epoch in range(0):

    model.train()

    # train = tqdm(train_loader)

    for data in train_loader:

        for key in data.keys():

            data[key] = data[key].to(device)

        outputs = model(**data)
        print(outputs)
        loss = outputs.loss # SequenceClassifierOutput 的 loss
        train.set_description(f'Epoch {epoch}')
        train.set_postfix({'Loss': loss.item()})
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    # 模型fine tune 完成
    # 驗證資料
    model.eval()
    # test = tqdm(test_loader)
    correct = 0
    for data in test_loader:
        for key in data.keys():
            data[key] = data[key].to(device)
        outputs = model(**data)
        _,predict_label = torch.max(outputs.logits,1) # 1 是 代表取col最大值
        correct += (predict_label==data['labels']).sum()
        test.set_description(f'Epoch {epoch}')
        test.set_postfix({'acc':'{:.4f}'.format(correct / len(test_set) * 100)})
    model.save_pretrained('model_{}'.format(epoch))

  0%|          | 0/9098 [00:00<?, ?it/s]

串流輸出內容已截斷至最後 5000 行。
        [-6.7287e-01, -1.2993e-01,  1.1753e-01, -3.2371e-01,  1.0047e+00,
          1.5445e+00, -1.1782e+00, -3.1304e-01],
        [-2.2944e+00,  1.8071e+00, -1.6645e+00, -1.0171e+00,  1.4205e+00,
         -1.2756e+00, -1.2875e+00,  2.5773e+00],
        [-2.6811e+00,  2.3908e+00, -1.9402e+00, -1.5071e+00,  2.7144e+00,
         -5.4187e-01, -1.6911e+00,  1.7098e+00],
        [-2.5043e+00, -2.2318e-01, -5.7364e-01, -1.8724e+00,  3.9804e+00,
          2.9317e-01, -1.5377e-01,  7.2100e-01],
        [-1.7449e+00, -1.6899e-01, -4.0628e-01, -1.2566e+00,  5.0578e-01,
          6.3270e+00, -1.2060e+00, -8.9701e-01],
        [-2.3667e+00,  2.8203e+00, -5.7506e-01, -1.1423e+00,  1.1359e+00,
         -5.0807e-01, -1.9847e+00,  7.6865e-01],
        [-6.8844e-01, -4.2733e-01,  1.2231e-01, -3.5238e-01,  1.4648e+00,
          3.4316e-01, -9.1254e-01,  1.0694e-01],
        [-1.8598e-01, -5.6997e-01,  6.4199e-01,  2.1539e-02,  6.7888e-01,
          7.2518e-01, -7.3786e-01, -4.9998e-

In [27]:
data[key].shape

torch.Size([32, 250])

In [34]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

結果紀錄
- Proprocessing + TFIDF(max=0.9, 3000, tette.tokenizer) + RF 0.36895
- TFIDF(3000) + RF 0.35301
- TFIDF(1100) + RF 0.34696

- Word2Vec + RF
  - Original Tokens : 0.32351
  - Preprocessing : 0.34046
  
- TFIDF(1000) + NN : 0.35494
